In [ ]:
class Classification:
    def __init__(self, classification_data):
        self.__vector = self.create_vector(classification_data)
        self.__classes = self.create_classes()
        
        self.__matrix = self.create_matrix()
        self.__set = self.create_set()
        
        self.__elements_number = len(self.__vector)
    
    def create_vector(self, classification_data):
    # TODO Create the vector representation from the pandas.DataFrame (single column object or data column with index column)
        pass
    
    def create_classes(self):
        # Creation of the index set, this indicates the classes exist within the
        # objects passed through the vector representation of the objects
        
        return set(self.__vector)    
    
    def create_matrix(self):
        # To get the indexes and initialize the matrix
        
        index = self.get_classes()
        matrix = []

        # For each of the classes, we identify from the vector what class
        # they belong to

        for i in self.__vector:
            row = []
            for j in index:
                row.append(1 if i == j else 0)
            matrix.append(row)
            
        return matrix
        
    def create_set(self):
        # Creation of the dictionary that indicates the classes and its objects
        
        set_rep = {i: set() for i in self.get_classes()}
        
        # For each element of the vector representation, we take the element as
        # the set it will be sent to and the index as the element label
        
        for i in range(len(self.__vector)):
            set_rep[self.__vector[i]].add(i)
        
        return set_rep
        
    def get_vector(self):
        return self.__vector
    
    def get_matrix(self):
        return self.__matrix
        
    def get_set(self):
        return self.__set
    
    def get_elements_number(self):
        return self.__elements_number
    
    def get_classes(self):
        return self.__classes
    
    def get_class_probability(self, class_id):
        # If the class ID is not in the classifier, the probability is 0,
        # otherwise, the probability is computed as the proportion of
        # instances
        
        if class_id not in self.get_classes():
            return 0
        
        return len(self.get_set()[class_id]) / self.get_elements_number()
    

In [ ]:
class BayesianClassifier:
  def __init__(self):
    pass

  def fit(self, attribute_data, objective_data, data_type):
    """
    ***Description***
  
    :params:
    attribute_data : Pandas.DataFrame
    A DataFrame containing the attributes of the patterns that will be used to learn the classes. Will have l columns and n rows.
    
    objective_data : Pandas.DataFrame
    A DataFrame containing a single column that indicates the class of the patterns. Must have n rows. 
  
    data_type : dict
    A dictionary whose length has to be the same as the attributes vector of the training dataframe (l entries). It indicates wether the data of a column is "continous", "integer" or "categoric".
    """
  
    """
    TODO :
    Las probabilidades a priori de cada clase en Y. Es decir, P(Y=y_i)
  
    Las funciones de masa o densidad de probabilidad condicionales p(X_j = x | Y = y_i), para cada j=1,...,L y y_i para i=1,...,M, donde L es la dimensión de los datos y M el número de clases.
  
    Para las variables categóricas debe usar la distribución Bernoulli (o en general distribución categórica).
  
    Para las variables enteras calcular la media y estimar el pdf de una distribución Poisson.
  
    Para las variables continuas calcular la media y la varianza de los datos para estimar el pdf de la normal, o estimar el KDE, de acuerdo con el parámetro de la clase.
  
    """
    pass
  
  def predict(self, x_predict):
    """
    ***Description***
  
    :params:
    x_pred : list / array / Pandas.Series
    An object that will contain the attributes vector of a pattern.
  
    :return:
    class : ????
    The class with the greatest likelihood according to the Naive Bayes Classifier.
    """
  
    """
    TODO
    Recibir un vector X_pred que debe tener el mismo número de columnas que X.
  
    Calcular el clasificador bayesiano ingenuo \PI_{j=1}^L p(X_j=x_j | Y=y_i)P(Y=y_i) para cada clase i=1,...,M y devolver el valor de y_i que lo maximiza.
    """
    pass

In [8]:
from ucimlrepo import fetch_ucirepo 

from pandas import concat

# fetch dataset 

heart_disease = fetch_ucirepo(id = 45) 
  
# data (as pandas dataframes)

x = heart_disease.data.features 
y = heart_disease.data.targets 
  
data = concat([y, x], axis = 1)
data

,num,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,2,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0
2,1,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0
3,0,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
4,0,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,1,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0
299,2,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0
300,3,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0
301,1,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0
